In [9]:
from googleapiclient.discovery import build
import sys
# Load custom scripts in reusable_code folder
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf

creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/')

In [10]:
bb_palette={'Union Blue':{'blue': 0.376470588235294,'green': 0.207843137254902,'red': 0.0313725490196078},
'Sunset Red':{'blue': 0.364705882352941,'green': 0.32156862745098,'red': 0.823529411764706},
'Dorset Blue':{'blue': 0.72156862745098,'green': 0.627450980392157,'red': 0.294117647058824},
'Apple':{'blue': 0.549019607843137,'green': 0.709803921568628,'red': 0.552941176470588},
'Orange':{'blue': 0.368627450980392,'green': 0.513725490196078,'red': 0.87843137254902},
'Apricot':{'blue': 0.364705882352941,'green': 0.701960784313725,'red': 0.909803921568627},
'Dorset Blue Highlight':{'blue': 0.901960784313726,'green': 0.792156862745098,'red': 0.36078431372549},
'Sunset Red Highlight':{'blue': 0.6,'green': 0.56078431372549,'red': 1},
'Union Blue Highlight':{'blue': 0.811764705882353,'green': 0.415686274509804,'red': 0},
'Raspberry':{'blue': 0.392156862745098,'green': 0.247058823529412,'red': 0.6},
'Dark Storm':{'blue': 0.137254901960784,'green': 0.125490196078431,'red': 0.117647058823529},
'Mild Storm':{'blue': 0.192156862745098,'green': 0.168627450980392,'red': 0.156862745098039},
'Light Storm':{'blue': 0.294117647058824,'green': 0.262745098039216,'red': 0.243137254901961},
'Dark Cloud':{'blue': 0.847058823529412,'green': 0.823529411764706,'red': 0.803921568627451},
'Light Cloud':{'blue': 0.956862745098039,'green': 0.929411764705882,'red': 0.909803921568627},
'White':{'blue': 1,'green': 1,'red': 1},}

In [11]:
# One off retrieval of SLIDES service
service = build('slides', 'v1', credentials=creds)
slides = service.presentations()

# One off retrieval of SHEETS service
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

In [14]:
slideID='1tRtUmxdgyQj9aEdAc6wNqMpFHjvWXbDvBrJGqFmGYdg'

In [15]:
slide_info=slides.get(presentationId=slideID).execute()

In [16]:
chart_dict={}
from pprint import pprint 
for i in slide_info['slides']:
    for j in i['pageElements']:
        if "sheetsChart" in j.keys():
            if j["sheetsChart"]["spreadsheetId"] in chart_dict.keys():
                chart_dict[j["sheetsChart"]["spreadsheetId"]].append(j["sheetsChart"]["chartId"])
            else:
                chart_dict[j["sheetsChart"]["spreadsheetId"]]=([j["sheetsChart"]["chartId"]])


In [17]:
chart_dict

In [36]:
def format_graphs(spreadsheet_id,chart_id,graph_title=None,left_axis_title=None,bottom_axis_title=None):
    
    sheet_info=sheet.get(spreadsheetId=spreadsheet_id).execute() # Get all info on the google sheet
    all_charts=[]
    for eachtab in sheet_info['sheets']:
        if 'charts' in eachtab.keys():
            for eachchart in eachtab['charts']:
                all_charts.append(eachchart)
            
    
    chart_spec=[x for x in all_charts if str(x['chartId'])==chart_id][0] # Full dictionary of First sheet, first chart
    new_chart_spec=chart_spec
    
            #set chart default font
    new_chart_spec['spec']['fontName']='Verdana'

    # Set chart title
    if graph_title:
        new_chart_spec['spec']['title']=graph_title

    # Set title font type, size and colour
    new_chart_spec['spec']['titleTextFormat']={
      "foregroundColor": bb_palette['Dark Storm'],
      "fontFamily": 'Verdana',
      "fontSize": 18,
      "bold": True,
      "italic": False
    }
    # Set title position/ alignment
    new_chart_spec['spec']['titleTextPosition']={'horizontalAlignment': 'CENTER'}

    # Set background colour
    new_chart_spec['spec']['backgroundColor']=bb_palette['White']

    if 'basicChart' in new_chart_spec['spec'].keys(): 
        # Set legend position
        new_chart_spec['spec']['basicChart']['legendPosition']='BOTTOM_LEGEND'

        
        
         # Set left axis
        Left_axis_index=[n for n,i in enumerate(new_chart_spec['spec']['basicChart']['axis']) if i['position']=='LEFT_AXIS'][0]

        # Axis title
        if left_axis_title:
            new_chart_spec['spec']['basicChart']['axis'][Left_axis_index]['title']=left_axis_title
        # Axis text format
        new_chart_spec['spec']['basicChart']['axis'][Left_axis_index]['format']={
          "foregroundColor": bb_palette['Dark Storm'],
          "fontFamily": 'Verdana',
          "fontSize": 14,
          "bold": False,
          "italic": True
        }
        
        if 'title' in new_chart_spec['spec']['basicChart']['axis'][Left_axis_index].keys():
            # Axis text alignment
            new_chart_spec['spec']['basicChart']['axis'][Left_axis_index]['titleTextPosition']={'horizontalAlignment': 'CENTER'}


            
            
            # Set bottom axis
        Bottom_axis_index=[n for n,i in enumerate(new_chart_spec['spec']['basicChart']['axis']) if i['position']=='BOTTOM_AXIS'][0]
        # Axis title
        if bottom_axis_title:
            new_chart_spec['spec']['basicChart']['axis'][Bottom_axis_index]['title']=bottom_axis_title

        # Axis text format
        new_chart_spec['spec']['basicChart']['axis'][Bottom_axis_index]['format']={
          "foregroundColor": bb_palette['Dark Storm'],
          "fontFamily": 'Verdana',
          "fontSize": 14,
          "bold": True,
          "italic": False
        }

        if 'title' in new_chart_spec['spec']['basicChart']['axis'][Bottom_axis_index].keys():
            # Axis text alignment
            new_chart_spec['spec']['basicChart']['axis'][Bottom_axis_index]['titleTextPosition']={'horizontalAlignment': 'CENTER'}

            
        # Colour series
        for n1,i in enumerate(new_chart_spec['spec']['basicChart']['series']):    
            i['color']=[bb_palette[x] for n,x in enumerate(bb_palette.keys()) if n==n1][0]
        
    data = {'requests': [
    {
        'updateChartSpec':{"chartId":new_chart_spec['chartId'],
                           "spec":new_chart_spec['spec']
                          }
    }
]}

    sheet.batchUpdate(spreadsheetId=spreadsheet_id, body=data).execute()


In [38]:
excl_list=[]
for k in chart_dict:
    for x in chart_dict[k]:
        if k not in excl_list:
            print(k)
            print(x)
            #format_graphs(k,str(x))
            try:
                format_graphs(k,str(x))
            except:
                print('Couldn''t amend graph')

In [41]:
sheet_info=sheet.get(spreadsheetId='18Jkn__U-YrQxWxepmUO2LDYefVriwOLynAyD5RmMzQE').execute() # Get all info on the google sheet
chart_spec=[x for x in sheet_info['sheets'][0]['charts'] if str(x['chartId'])==chart_id][0] # Full dictionary of First sheet, first chart

In [42]:
sheet_info=sheet.get(spreadsheetId='18Jkn__U-YrQxWxepmUO2LDYefVriwOLynAyD5RmMzQE').execute() # Get all info on the google sheet
all_charts=[]
for eachtab in sheet_info['sheets']:
    if 'charts' in eachtab.keys():
        for eachchart in eachtab['charts']:
            all_charts.append(eachchart)


chart_spec=[x for x in all_charts if str(x['chartId'])=='1473784709'][0] # Full dictionary of First sheet, first chart

In [43]:
chart_spec